In [ ]:
/**
  OK cool, we are switching over to JavaScript / NodeJS now.
  If you have only Python installed in Jupyter notebooks, you can read more here:
  https://github.com/notablemind/jupyter-nodejs
  
  We need the client-side JavaScript to generate a public-private keypair
  Then encrypt the client's geolocation
  Then send it up to the server.
  
  This looks like the right library:
  https://github.com/hardbyte/paillier.js
  On GitHub it is called paillier.js, but the node module is 'homomorphicjs'
  
  You might wonder why not NodeJS on the server, then?
  I couldn't solve how to make the JS library do addition and multiplication
  It does everything that we need, though.
*/

In [2]:
/*
  When we work with the encrypted numbers, they are very big.
  JavaScript only supports up to 53-bit integers http://2ality.com/2012/07/large-integers.html
  
  This library has jsbn as a dependency for handling big numbers. I did some stuff
  to browserify it. But we are going to work with the NodeJS version.
*/

var phe = require('../node_modules/homomorphicjs');

// Let's look at the keypair
var keypair = phe.generate_paillier_keypair(128);
console.log(keypair);

{"public_key":{"g":"241633540509182103983082950696774583572","n":"241633540509182103983082950696774583571"},"private_key":{"lambda":"241633540509182103951938125174484827944","mu":"140422264154311621432317355421168792233"},"n_length":128}


In [6]:
// Now let's get the latitude and longitude
// In the browser this will work, but in NodeJS mode we will not know

if (typeof navigator !== 'undefined') {
  navigator.geolocation.getCurrentPosition(function(position) {
    console.log(position);
    /* {
          coords: {
            latitude: ##, longitude: ##, accuracy: ##
          },
          timestamp: ##
       } */
  }, function(errorResponse) {
    console.log(errorResponse); 
  });
} else {
    // NodeJS version
}

In [ ]:
// For the notebook, let's assume that we got these coordinates:

var latitude = 19.7221702;
var longitude = -155.0904464;

// Round them to 3 decimal places like we did on the server
// Handle negative latitude and longitude in this phase, too (0-360, not -180 to +180)
latitude = Math.round(latitude * 1000) + 90000;
longitude = Math.round(longitude * 1000) + 180000;


hideLatitude = keypair.public_key.raw_encrypt(latitude);
hideLongitude = keypair.public_key.raw_encrypt(longitude);

// Unfortunately this totally broke Jupyter notebook. I promise that it works in the browser

// Then post keypair.public_key.g,  keypair.public_key.n, latitude, and longitude to the server

In [ ]:
// Interpreting the response:

var northOffset = kp.private_key.raw_decrypt(response.northOffset);
var southOffset = kp.private_key.raw_decrypt(response.southOffset);
var eastOffset = kp.private_key.raw_decrypt(response.eastOffset);
var westOffset = kp.private_key.raw_decrypt(response.westOffset);

// We check if x and y match by comparing to north-south and east-west bounds
// Inside the geo-area, one of the pair should be positive, and the other should be negative
// If the number is negative, it will appear very large on our side.

var ymatch = 
    (Math.log(northOffset) > 50 && Math.log(southOffset) < 50) ||
    (Math.log(northOffset) < 50 && Math.log(southOffset > 50));
var xmatch =
    (Math.log(eastOffset) > 50 && Math.log(westOffset) < 50) ||
    (Math.log(eastOffset) < 50 && Math.log(westOffset > 50));

if (xmatch && ymatch) { console.log('success!') }

/*
You might wonder why we generated the key on the client, and let the client decide.
Why not generate the key on the server, let the client do the math on their coordinates, and
then have the server determine if the client is within those bounds?

- First, I don't have the JS working to do the addition and multiplication
- Second, a sneaky server might start with a geobox around 0,0 and based on the user's answers,
  test several different geofences to narrow down the client's location.  If we make a standard
  for encrypting coordinates, it ought to be controlled and evaluated by the client.
*/